In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. 주거 정보 구하기

# 2. 버스 정류장 좌표를 통해 주변 시설 구하기

>## 2.1데이터 불러오기

In [3]:
bus = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/제출용데이터dir/2022년_전국버스정류장 위치정보_데이터.csv')
bus.head(2)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산


>## 2.2 카카오 api를 통하여, 천안시 버스정류장을 필터링하기
천안시의 버스정류장을 고려하기 위해서, 천안시의 버스정류장 데이터를 필터링한다.

제공된 버스정류장의 데이터에는 위도, 경도 좌표에 부여된 도시명과 맞지 않는 부분이 있기 때문에,

카카오 api를 활용하여, 도로명 주소를 구하고 도로명주소를 활용하여 버스 정류장이 천안시에만 있는 데이터만을 추출한다.


In [5]:
addressCopy = bus.copy()
addressCopy = addressCopy.head()
apiKey='1d49e941371e685a965fba06520cda68'
headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

addressCopy['coord2address'] = None

cnt=0 
for idx in tqdm(range(len(addressCopy))):
  try:

    x = addressCopy.at[idx,'경도']
    y =  addressCopy.at[idx,'위도']
    searching ='x={x}&y={y}'.format(x=str(x),y=str(y))
    url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    # print(result)
    documents = result['documents']
    
    addressCopy.at[idx,'coord2address'] = documents[0]
  except:
    cnt+=1
    if cnt%50==0:
      print('쿼리값을 못찾는중:',cnt) 

    pass

addressCopy

100%|██████████| 5/5 [00:00<00:00,  8.62it/s]


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."


In [6]:
# 제공된 버스정류장의 데이터에선 정류장번호가 ASB214010001 인 데이터의 도시명은 아산시라고 되어있지만, 카카오 api를 통해 나온 도로명 주소는 평택시로 되어있다.
# 구글 맵을 통해, 위도 경도 좌표를 넣어보면, 구글 맵또한 평택시라고 명시되어 있다.
addressCopy.loc[0,'coord2address']

# '시' 를 구분하는 region_2depth_name 데이터를 통해, 제공된 버스 정류장 데이터에 새로운 칼럼을 부여하여,
# region_2depth_name 칼럼을 통해, 천안시인 데이터만을 필터링한다.
addressCopy['region_2depth_name'] = addressCopy['coord2address'].apply(lambda elm : elm.get('address').get('region_2depth_name'))
addressCopy

{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,region_2depth_name
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시


In [9]:
# 천안시의 데이터만 필터링한 후 저장하기
filtered_addressCopy = addressCopy[addressCopy['region_2depth_name']=='천안시']
filtered_addressCopy.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/제출용데이터dir/bus_coord2address.pickle')

>## 2.3 카카오 api를 통해 얻은 도로명 주소를 통해 반경 0.5km 이내에 있는 시설 종합하기

In [10]:
# ToDo. 도로명주소 -> 반경 0.5km 이내에 있는 데이터를 긁는 코드 필요

In [18]:
bus_byApi = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/제출용데이터dir/Bus_around_500_08231838 복사본.pickle')
bus_byApi.head(3)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,0,8,0,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,0,3,0,0,0
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# 모든 카테고리의 수를 종합한 칼럼 category_sum 칼럼은 추후 score 칼럼을 계산할 때 사용된다
bus_byApi['category_sum'] = bus_byApi.loc[:,'MT1':].sum(axis=1)
bus_byApi.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,8,0,1,0,34
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,3,0,0,0,24


# 3. 버스 정류장 - 공원 사이의 거리 구하기

버스 정류장과 공원 사이의 거리를 구하여, 구간 별로 나눈 범위 중 몇개가 있는지 구한다

In [27]:
# ToDo. 거리 계산한 코드 필요

In [26]:
bus_park = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/천안시 공원정보/버스정류장과공원사이의거리.pickle')
bus_park

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


* class_A : 버스정류장과 타겟과의 거리가 0이상 0.5 이하에 속하는 것

* class_B : 버스정류장과 타겟과의 거리가 0.5초과 1 이하에 속하는 것

        (거리 단위는 km)

In [28]:
# 버스 정류장과 공원 사이의 거리가 0<=distance<=0.5, 0.5<distance<=1 의 갯수를 각각 구한다.
bus_park_class_A = bus_park[(0<=bus_park['distance'])&(bus_park['distance']<=0.5)]
bus_park_class_B = bus_park[(0.5<bus_park['distance'])&(bus_park['distance']<=1)]
bus_park_class_A.head(2)
bus_park_class_B.head(2)

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
1493,ASB285000099,삼풍아파트,"(36.9125166, 127.1309215)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.317816
2933,ASB285000165,쌍용동일아파트,"(36.7848, 127.11328)",쌍용10공원,"(36.78655184, 127.1168556)",44133-00077,0.373709


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
785,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.568513
962,ASB285000096,성신초등학교,"(36.9176694, 127.1277947)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.842896


In [29]:
# 공원거리구분_A : 0.5<distance<=1 
class_A_df= pd.DataFrame(bus_park_class_A['정류장번호'].value_counts()).reset_index()
class_A_df.columns =['정류장번호','공원거리구분_A']
class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
class_A_df.reset_index(drop=True,inplace=True)
class_A_df

# 공원거리구분_B : 0.5<distance<=1 


class_B_df= pd.DataFrame(bus_park_class_B['정류장번호'].value_counts()).reset_index()
class_B_df.columns =['정류장번호','공원거리구분_B']
class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
class_B_df.reset_index(drop=True,inplace=True)
class_B_df

,정류장번호,공원거리구분_A
0,ASB285000099,1
1,ASB285000165,1
2,ASB285000287,1
3,ASB285000649,2
4,ASB285000650,2
...,...,...
781,GGB277102211,1
782,GGB285000034,1
783,GGB285500010,4
784,GGB285500020,6


,정류장번호,공원거리구분_B
0,ASB285000095,1
1,ASB285000096,1
2,ASB285000097,1
3,ASB285000098,1
4,ASB285000100,1
...,...,...
1008,GGB285500010,4
1009,GGB285500020,8
1010,GGB285500030,8
1011,GGB288500130,8


In [31]:
bus_station= bus_byApi.copy()
bus_station_merge_class_A = pd.merge(bus_station,class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print(len(bus_station)==len(bus_station_merge_class_A))

bus_station_merge_class_AB = pd.merge(bus_station_merge_class_A,class_B_df, on='정류장번호',how='left')


print(len(bus_station)==len(bus_station_merge_class_AB))

True
True


In [32]:
bus_station_merge_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].fillna(0)
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].fillna(0)

bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].astype('int')
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].astype('int')

bus_station_merge_class_AB.head(1)


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,34,NaN,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,34,0,0


# 4. 버스 정류장과 거주지, 회사의 갯수 구하기

위의 작업과 마찬가지로, 버스정류장과 거주지,회사간의 거리를 구하여 그 거리를 구간별로 나눠 갯수를 구하는 작업을 진행한다.


In [33]:
bus_CH = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/거리계산/bus_Around_company_house.pickle')
bus_CH.head(2)

,정류장번호,정류장명,coord2address,주변시설의총합,location,이름,byAPI,house_loc,종류,distance
0,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.429916
1,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.335470


In [35]:
# A, B 로 구간 나누기
bus_CH_A = bus_CH[(0<=bus_CH['distance'])&(bus_CH['distance']<=0.5)]
bus_CH_B = bus_CH[(0.5<bus_CH['distance'])&(bus_CH['distance']<=1)]



In [36]:
# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_CH_class_A_df= pd.DataFrame(bus_CH_A['정류장번호'].value_counts()).reset_index()
bus_CH_class_A_df.columns =['정류장번호','CH거리구분_A']
bus_CH_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_A_df.reset_index(drop=True,inplace=True)
bus_CH_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5< distance<=1
bus_CH_class_B_df= pd.DataFrame(bus_CH_B['정류장번호'].value_counts()).reset_index()
bus_CH_class_B_df.columns =['정류장번호','CH거리구분_B']
bus_CH_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_B_df.reset_index(drop=True,inplace=True)
bus_CH_class_B_df.head(2)

,정류장번호,CH거리구분_A
0,ASB285000077,5
1,ASB285000078,11


,정류장번호,CH거리구분_B
0,ASB285000077,25
1,ASB285000078,15


In [37]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_class_AB.copy()

bus_station_merge_CH_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_CH_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_A))

bus_station_merge_CH_class_AB = pd.merge(bus_station_merge_CH_class_A,bus_CH_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_AB))

bus_station_merge_CH_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].fillna(0)
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].fillna(0)

bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].astype('int')
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].astype('int')

# 병합된 데이터 셋
bus_station_merge_CH_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,34,0,0,5.0,25.0


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,34,0,0,5,25


# 5. 버스 정류장과 하천과의 거리 갯수 구하기
위의 작업들과 동일하며, 버스정류장과 하천과의 거리를 구하여 구간별로 나눠 갯수를 센다

In [39]:
bus_stream = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/하천정보/버스정류장과하천과의거리.pickle')
bus_stream.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014


In [40]:
bus_stream_A  = bus_stream[(0<=bus_stream['distance'])&(bus_stream['distance']<=0.5)]
bus_stream_B  = bus_stream[(0.5<bus_stream['distance'])&(bus_stream['distance']<=1)]
bus_stream_A.head(2)
bus_stream_B.head(2)


# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_stream_class_A_df= pd.DataFrame(bus_stream_A['정류장번호'].value_counts()).reset_index()
bus_stream_class_A_df.columns =['정류장번호','하천거리구분_A']
bus_stream_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_A_df.reset_index(drop=True,inplace=True)
bus_stream_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5<distance<=1
bus_stream_class_B_df= pd.DataFrame(bus_stream_B['정류장번호'].value_counts()).reset_index()
bus_stream_class_B_df.columns =['정류장번호','하천거리구분_B']
bus_stream_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_B_df.reset_index(drop=True,inplace=True)
bus_stream_class_B_df.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
7736,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.398270
37124,ASB285000094,성환8리,"(36.9214794, 127.1225049)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.421355


,정류장번호,정류장명,location,A1,하천좌표,distance
17532,ASB285000078,송덕,"(36.92378302, 127.11897568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.813301
46920,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.613733


,정류장번호,하천거리구분_A
0,ASB285000077,1
1,ASB285000094,1


,정류장번호,하천거리구분_B
0,ASB285000078,1
1,ASB285000095,1


In [41]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_CH_class_AB.copy()

bus_station_merge_stream_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_stream_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_A))

bus_station_merge_stream_class_AB = pd.merge(bus_station_merge_stream_class_A,bus_stream_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_AB))

bus_station_merge_stream_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].fillna(0)
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].fillna(0)

bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].astype('int')
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].astype('int')

bus_station_merge_stream_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,34,0,0,5,25,1.0,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,34,0,0,5,25,1,0


In [42]:
# 최종적으로 사용하게 될 데이터프레임
bus_station_merge_stream_class_AB.head()


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,34,0,0,5,25,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,24,0,0,11,15,0,1
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,6,0,0,12,22,0,0
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,0,0,30,0,0,5,24,1,0
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,16,0,560,0,1,11,16,0,1


# Score 계산하기

버스정류장의 반경 2km 이내에 있는 각 카테고리(마트,편의점,병원,,)의 수 를 집계하여

구분 A의 값은 2배의 값을 곱하여 더하고,

구분 B의 값은 자기 자신의 값을 더하여 score 칼럼을 구한다.

(score 칼럼 : 버스정류장의 중요도를 나타내는 정도)

In [43]:
score_df = bus_station_merge_stream_class_AB.copy()

In [48]:
# score 칼럼 구하기
score_df['score'] = score_df['category_sum'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
# sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
# sorted_score_df
score_df.head(2)


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,1,0,34,0,0,5,25,1,0,71
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,24,0,0,11,15,0,1,62


# 6. 클러스터링 진행하기